In [ ]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib import cm
import scipy.stats as stats


In [ ]:
bucket = os.getenv("WORKSPACE_BUCKET")
bucket

In [ ]:
%%writefile ~/aou_dsub.bash

#!/bin/bash

function aou_dsub () {

  # Get a shorter username to leave more characters for the job name.
  local DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

  # For AoU RWB projects network name is "network".
  local AOU_NETWORK=network
  local AOU_SUBNETWORK=subnetwork

  dsub \
      --provider google-cls-v2 \
      --user-project "${GOOGLE_PROJECT}"\
      --project "${GOOGLE_PROJECT}"\
      --network "${AOU_NETWORK}" \
      --subnetwork "${AOU_SUBNETWORK}" \
      --service-account "$(gcloud config get-value account)" \
      --user "${DSUB_USER_NAME}" \
      --regions us-central1 \
      --boot-disk-size 55 \
      --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
      "$@"
}

In [ ]:
%%bash
chmod +x ~/aou_dsub.bash
echo source ~/aou_dsub.bash >> ~/.bashrc

In [ ]:
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env USER_NAME={USER_NAME}

In [ ]:
phenos = []
with open('phenos.list','rt') as inpu:
    for i in inpu:
        if i =="\n":
            continue
        phenos.append(i.replace('\n',''))

In [ ]:
pdict = {'continuous-50-both_sexes-irnt':"Height",'continuous-23104-both_sexes-irnt':"BMI",'continuous-30050-both_sexes-irnt':"MCH",'continuous-30040-both_sexes-irnt':"MCV",'continuous-30140-both_sexes-irnt':"Neutrophil_count",'continuous-30000-both_sexes-irnt':"WBC_count"}


In [ ]:
samples = np.array(pd.read_csv(f"{bucket}/panukbb/data/aou_v7_testPops_chr1.fam",sep="\t",header=None)[1])


In [ ]:
df = pd.read_csv(f"{bucket}/panukbb/phenos/0723_quant_combined_panukbb_v7_rint.tsv",sep="\t")
df = df[df["IID"].isin(samples)]

In [ ]:
h2s = pd.read_csv(f"ldsc_rg_EUR.csv",sep=",")

In [ ]:
phenodf = pd.read_csv("gs://fc-secure-06f42177-4b29-4956-88a8-88ede84cb2ab/panukbb/phenos/0723_panukbb_training_phenos.tsv",sep="\t")


In [ ]:
create = False

# Rescaling 

In [ ]:
minall = 1
maxall = 0

for anc in ["","_EURPRS"]:
    for i in range(len(phenos)):
        df = pd.read_csv(f"{bucket}/panukbb/individualPRS/merged_result{anc}/{phenos[i]}_accuracy_zscore.tsv",sep="\t",header=None)
        minlocal = min(df[2])
        maxlocal = max(df[2])
        if minlocal<minall:
            minall = minlocal
        if maxlocal>maxall:
            maxall = maxlocal

In [ ]:
for anc in ["","_EURPRS"]:
    for i in range(len(phenos)):
        df = pd.read_csv(f"{bucket}/panukbb/individualPRS/merged_result{anc}/{phenos[i]}_accuracy_zscore.tsv",sep="\t",header=None)
        df[2] = (df[2]-minall)/(maxall-minall)
        df.to_csv(f"{bucket}/panukbb/individualPRS/merged_result{anc}/{phenos[i]}_rescaled.tsv",sep="\t",index=None,header=None)


In [ ]:
maxall, minall

# Supplementary

In [ ]:
def plot_all(ax,x,y,groups,groupind,label="Multi",marker='o',veritcal="top",xc=0.99,yc=0.9,legend=True,line="--",color="black",text=True):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    colors = ["#941494","#ED1E24","#FF9912","#108C44","#6AA5CD","#33CC33"]

    coefficients = np.polyfit(x, y, 1)
    slope, intercept = coefficients
    polynomial = np.poly1d(coefficients)
    x_fit = np.linspace(min(x), max(x), 100)
    y_fit = polynomial(x_fit)
    index = 0
    for name, group in groups:
        #group = group.iloc[:int(len(group)*0.01)]
        if legend:
            ax.plot(group["euc_dist"], group[groupind], marker=marker, linestyle='', ms=5, label=f'{name}_PRS_{label}',alpha=0.15,c=colors[index])
        else:
            ax.plot(group["euc_dist"], group[groupind], marker=marker, linestyle='', ms=5, alpha=0.15,c=colors[index])
        index+=1
    ax.plot(x_fit, y_fit, line,color=color, label=f'{label} Line of Best Fit',zorder=10)
    corr = stats.pearsonr(x, y).statistic
    if text:
        ax.text(xc, yc, f'PRS {label} R: {corr:.3f}, y = {intercept:.2f} - {-slope:.2f}x',
                 horizontalalignment='left',
                 verticalalignment=veritcal,
                 transform = ax.transAxes)


def sample_legend(ax,xb,yb):
    
    labels = ['AMR', 'EUR', 'AFR', 'CSA', 'EAS', 'MID',"PRS Multi (AoU)","PRS EUR (AoU)"]
    colors = ["#ED1E24","#6AA5CD","#941494","#FF9912","#108C44","#33CC33","black","black"]
    markers = ['s', 's', 's', 's', 's', 's',"o","D"]
    lines = ['--','-']
    lcolors = ["blue","black"]
    llabels = ["PRS Multi R","PRS EUR R"]
    
    handles = [plt.Line2D([0], [0], marker=marker, color='w', markerfacecolor=color, markersize=10, label=label)
           for marker, color, label in zip(markers, colors, labels)]
    
    
    line_handles = [plt.Line2D([0], [0], color=line_color, linestyle=linestyle, linewidth=2, label=line_label)
                for line_color, linestyle, line_label in zip(lcolors, lines, llabels)]
    
    handles+=line_handles
    
    ax.legend(handles=handles,  bbox_to_anchor=(xb,yb),fontsize=13,frameon=False)


In [ ]:
phenos = []
with open('phenos.list','rt') as inpu:
    for i in inpu:
        if i =="\n":
            continue
        phenos.append(i.replace('\n',''))
pdict = {'continuous-30000-both_sexes-irnt':"WBC_count",'continuous-30140-both_sexes-irnt':"Neutrophil_count",'continuous-23104-both_sexes-irnt':"BMI",'continuous-30040-both_sexes-irnt':"MCV",'continuous-30050-both_sexes-irnt':"MCH",'continuous-50-both_sexes-irnt':"Height"}
adict = {"0424_EURtrain_proj_euc_test.tsv":"EUR"}
pvalues = ["White blood cell count","Neutrophil count","BMI","MCV","MCH","Height"]

def sample_legend(ax,xb,yb):
    
    labels = ['AMR', 'EUR', 'AFR', 'CSA', 'EAS', 'MID',"PRS EUR (AoU)"]
    colors = ["#ED1E24","#6AA5CD","#941494","#FF9912","#108C44","#33CC33","black"]
    markers = ['s', 's', 's', 's', 's', 's',"D"]
    lines = ['-']
    lcolors = ["black"]
    llabels = ["PRS EUR R"]
    
    handles = [plt.Line2D([0], [0], marker=marker, color='w', markerfacecolor=color, markersize=10, label=label)
           for marker, color, label in zip(markers, colors, labels)]
    
    
    line_handles = [plt.Line2D([0], [0], color=line_color, linestyle=linestyle, linewidth=2, label=line_label)
                for line_color, linestyle, line_label in zip(lcolors, lines, llabels)]
    
    handles+=line_handles
    
    ax.legend(handles=handles,  bbox_to_anchor=(xb,yb),fontsize=13,frameon=False)

In [ ]:
accs = []
for a in adict.keys():
    
    fig, ax = plt.subplots(6,1,figsize=(10,15))

    for p in range(len(pdict)):

        
        
        
        df = pd.read_csv(f"gs://fc-secure-06f42177-4b29-4956-88a8-88ede84cb2ab/panukbb/individualPRS/{a}",sep="\t")
        acc = pd.read_csv(f"{bucket}/panukbb/individualPRS/merged_result_EURPRS/{list(pdict.keys())[p]}_rescaled.tsv",sep="\t",header=None)
        acc = acc.rename(columns={0:"IID"})
        accs.append(acc)
        df = pd.merge(acc,df,on="IID",how="left")
        groups = df.groupby('Pop')
        
        
        plot_all(ax[p],df["euc_dist"], acc[2],groups,2,label="EUR",marker='D',veritcal="bottom",xc=0.01,yc=0,line="-",color="black")
        
        
        ax[p].set_title(pvalues[p])

    #ax[1].legend(bbox_to_anchor=(1.28,1.5))
    sample_legend(ax[2],1.28,0.5)
    ax[5].set_xlabel("Genetic distance from training population (AoU EUR-ancestry)",fontsize=15)
    fig.text(0.06, 0.5, 'Individual-level accuracy', va='center', rotation='vertical',fontsize=15)
    #fig.subplots_adjust(hspace=0.3)
    #fig.tight_layout()
    ax[0].set_xticks([])
    ax[1].set_xticks([])
    ax[2].set_xticks([])
    ax[3].set_xticks([])
    ax[4].set_xticks([])
    plt.savefig(f"SFIG8_indivacc_EUR",bbox_inches='tight',dpi=600)
    plt.show()

# Plot

In [ ]:
def plot_all(ax,x,y,groups,groupind,label="Multi",marker='o',veritcal="top",xc=0.99,yc=0.9,legend=True,line="--",color="black",text=True):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    colors = ["#941494","#ED1E24","#FF9912","#108C44","#6AA5CD","#33CC33"]

    coefficients = np.polyfit(x, y, 1)
    slope, intercept = coefficients
    print(slope,intercept)
    polynomial = np.poly1d(coefficients)
    x_fit = np.linspace(min(x), max(x), 100)
    y_fit = polynomial(x_fit)
    index = 0
    for name, group in groups:
        #group = group.iloc[:int(len(group)*0.01)]
        if legend:
            ax.plot(group["euc_dist"], group[groupind], marker=marker, linestyle='', ms=5, label=f'{name}_PRS_{label}',alpha=0.15,c=colors[index])
        else:
            ax.plot(group["euc_dist"], group[groupind], marker=marker, linestyle='', ms=5, alpha=0.15,c=colors[index])
        index+=1
    ax.plot(x_fit, y_fit, line,color=color, label=f'{label} Line of Best Fit',zorder=10)
    corr = stats.pearsonr(x, y).statistic
    pv = stats.pearsonr(x, y).pvalue
    print(pv)
    if text:
        ax.text(xc, yc, f'PRS {label} R: {corr:.3f}, y = {intercept:.2f} - {-slope:.2f}x',
                 horizontalalignment='left',
                 verticalalignment=veritcal,
                 transform = ax.transAxes)
    return slope,intercept,corr,pv
    
def plot_mean(groups,groupind,ax,marker='o'):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    index=0
    for name, group in groups:
        dist = np.mean(group["euc_dist"])
        uncer = np.mean(group[groupind])
        ax.errorbar(dist, uncer, yerr = np.std(group[groupind]), fmt=marker, capsize=5,c=colors[index])
        index+=1
        
def plot_meanstd(groups,groupind,ax,marker="o"):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    index = 0
    for name, group in groups:
        #group = group.iloc[:int(len(group)*0.01)]
        dist = group["euc_dist"]
        ax.errorbar(dist, group[groupind[0]], yerr = group[groupind[1]], fmt=marker, capsize=5,c=colors[index],alpha=0.25)
        index+=1
        

def sample_legend(ax,xb,yb):
    
    labels = ['AMR', 'EUR', 'AFR', 'CSA', 'EAS', 'MID',"PRS Multi (AoU)","PRS EUR (AoU)"]
    colors = ["#ED1E24","#6AA5CD","#941494","#FF9912","#108C44","#33CC33","black","black"]
    markers = ['s', 's', 's', 's', 's', 's',"o","D"]
    lines = ['--','-']
    lcolors = ["blue","black"]
    llabels = ["PRS Multi R","PRS EUR R"]
    
    handles = [plt.Line2D([0], [0], marker=marker, color='w', markerfacecolor=color, markersize=10, label=label)
           for marker, color, label in zip(markers, colors, labels)]
    
    
    line_handles = [plt.Line2D([0], [0], color=line_color, linestyle=linestyle, linewidth=2, label=line_label)
                for line_color, linestyle, line_label in zip(lcolors, lines, llabels)]
    
    handles+=line_handles
    
    ax.legend(handles=handles,  bbox_to_anchor=(xb,yb),fontsize=13,frameon=False,ncol=5)


In [ ]:
phenos = []
with open('phenos.list','rt') as inpu:
    for i in inpu:
        if i =="\n":
            continue
        phenos.append(i.replace('\n',''))
pdict = {'continuous-23104-both_sexes-irnt':"BMI",'continuous-30040-both_sexes-irnt':"MCV",'continuous-30000-both_sexes-irnt':"WBC_count",'continuous-30140-both_sexes-irnt':"Neutrophil_count"}
adict = {"0624_trainPops_gd.tsv":"Meta"}
pvalues = ["BMI","MCV","White blood cell count","Neutrophil count"]

In [ ]:
accs = []
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
table = []
for a in adict.keys():
    
    fig, ax = plt.subplots(4,1,figsize=(10,14))
    #plt.rcParams['text.usetex'] = False
    #plt.rcParams['font.family'] = ['sans-serif']
    #plt.rcParams['font.sans-serif'] = ['Arial'] + plt.rcParams['font.sans-serif']
    for p in range(len(pdict)):

        
        df = pd.read_csv(f"gs://fc-secure-06f42177-4b29-4956-88a8-88ede84cb2ab/panukbb/individualPRS/{a}",sep="\t")
        acc = pd.read_csv(f"{bucket}/panukbb/individualPRS/merged_result/{list(pdict.keys())[p]}_rescaled.tsv",sep="\t",header=None)
        acc = acc.rename(columns={0:"IID"})
        accs.append(acc)
        df = pd.merge(acc,df,on="IID",how="left")
        groups = df.groupby('Pop')

        # Plot
        
        
        result = plot_all(ax[p],df["euc_dist"], acc[2],groups,2,label="Multi",marker='o',veritcal="bottom",xc=0.01,yc=0.1,line="--",color="blue")
        table.append(list(result)+[pvalues[p],"Multi"])
        
        
        df = pd.read_csv(f"gs://fc-secure-06f42177-4b29-4956-88a8-88ede84cb2ab/panukbb/individualPRS/{a}",sep="\t")
        acc = pd.read_csv(f"{bucket}/panukbb/individualPRS/merged_result_EURPRS/{list(pdict.keys())[p]}_rescaled.tsv",sep="\t",header=None)
        acc = acc.rename(columns={0:"IID"})
        accs.append(acc)
        df = pd.merge(acc,df,on="IID",how="left")
        groups = df.groupby('Pop')
        
        
        result = plot_all(ax[p],df["euc_dist"], acc[2],groups,2,label="EUR",marker='D',veritcal="bottom",xc=0.01,yc=0,line="-",color="black")
        table.append(list(result)+[pvalues[p],"EUR"])
        
        ax[p].set_title(pvalues[p])
        ax[p].set_ylim([-0.05, 1.05])

    #ax[1].legend(bbox_to_anchor=(1.28,1.5))
    sample_legend(ax[3],1,-0.35)
    #ax[3].set_xlabel("Genetic distance from training population (AoU multi-ancestry meta-analysis)",fontsize=15)
    fig.text(0.06, 0.48, 'Individual-level accuracy', va='center', rotation='vertical',fontsize=15)
    fig.text(0.1, 0.08, "Genetic distance from training population (AoU multi-ancestry meta-analysis)",va='center',fontsize=15)
    #fig.delaxes(ax[3])
    #fig.subplots_adjust(hspace=0.3)
    ax[0].tick_params(labelbottom=False)    
    
    ax[1].tick_params(labelbottom=False)    
    ax[2].tick_params(labelbottom=False)    
    plt.subplots_adjust(hspace=0.7)
    #fig.tight_layout()
    plt.savefig(f"fig4_v4",bbox_inches='tight',dpi=600)
    plt.close()

In [ ]:
phenos = []
with open('phenos.list','rt') as inpu:
    for i in inpu:
        if i =="\n":
            continue
        phenos.append(i.replace('\n',''))
pdict = {'continuous-50-both_sexes-irnt':"Height",'continuous-30050-both_sexes-irnt':"MCH"}
adict = {"0624_trainPops_gd.tsv":"Meta"}
pvalues = ["Height","MCH"]


In [ ]:
accs = []
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for a in adict.keys():
    
    fig, ax = plt.subplots(2,1,figsize=(10,7))
    #plt.rcParams['text.usetex'] = False
    #plt.rcParams['font.family'] = ['sans-serif']
    #plt.rcParams['font.sans-serif'] = ['Arial'] + plt.rcParams['font.sans-serif']
    for p in range(len(pdict)):

        
        df = pd.read_csv(f"gs://fc-secure-06f42177-4b29-4956-88a8-88ede84cb2ab/panukbb/individualPRS/{a}",sep="\t")
        acc = pd.read_csv(f"{bucket}/panukbb/individualPRS/merged_result/{list(pdict.keys())[p]}_rescaled.tsv",sep="\t",header=None)
        acc = acc.rename(columns={0:"IID"})
        accs.append(acc)
        df = pd.merge(acc,df,on="IID",how="left")
        groups = df.groupby('Pop')

        # Plot
        
        
        result = plot_all(ax[p],df["euc_dist"], acc[2],groups,2,label="Multi",marker='o',veritcal="bottom",xc=0.01,yc=0.1,line="--",color="blue")
        table.append(list(result)+[pvalues[p],"Multi"])
        
        
        df = pd.read_csv(f"gs://fc-secure-06f42177-4b29-4956-88a8-88ede84cb2ab/panukbb/individualPRS/{a}",sep="\t")
        acc = pd.read_csv(f"{bucket}/panukbb/individualPRS/merged_result_EURPRS/{list(pdict.keys())[p]}_rescaled.tsv",sep="\t",header=None)
        acc = acc.rename(columns={0:"IID"})
        accs.append(acc)
        df = pd.merge(acc,df,on="IID",how="left")
        groups = df.groupby('Pop')
        
        
        result = plot_all(ax[p],df["euc_dist"], acc[2],groups,2,label="EUR",marker='D',veritcal="bottom",xc=0.01,yc=0,line="-",color="black")
        table.append(list(result)+[pvalues[p],"EUR"])
        
        ax[p].set_title(pvalues[p])
        ax[p].set_ylim([-0.05, 1.05])

    #ax[1].legend(bbox_to_anchor=(1.28,1.5))
    sample_legend(ax[1],1,-0.35)
    #ax[3].set_xlabel("Genetic distance from training population (AoU multi-ancestry meta-analysis)",fontsize=15)
    fig.text(0.06, 0.48, 'Individual-level accuracy', va='center', rotation='vertical',fontsize=15)
    fig.text(0.1, 0.05, "Genetic distance from training population (AoU multi-ancestry meta-analysis)",va='center',fontsize=15)
    #fig.delaxes(ax[3])
    #fig.subplots_adjust(hspace=0.3)
    ax[0].tick_params(labelbottom=False)      
    plt.subplots_adjust(hspace=0.7)
    #fig.tight_layout()
    plt.savefig(f"indivPRSacc_sup4",bbox_inches='tight',dpi=600)
    plt.close()
    

In [ ]:
table

In [ ]:
pd.DataFrame(table,columns=["slope","intercept","R","P","trait","training pop"]).to_csv("figure_params.tsv",index=None,sep="\t")